# STK-IN4300 - Oblig 1 - Kristian Wold

In collaboration with Lasse Lorentz Braseth

The aim of this analysis is to use Lasso regression to predict a response from multivariate data. CV was used to select the best penatly $\lambda$, and the R2-score was calculated on a independant test set to assess the goodness of the model. Bootstrap sampling was then used to establish confidence intervals and p-values for the resulting coefficients of the best model. 

We chose to use the dataset "Energy efficiency Data Set". The relevant response is "heating load", that is, the efficiency at which one is able to heat a room. In this analysis, we use only the features included in the data set, omitting derived features such as higher orders and interactions. Features X1 through X8 correspond to Relative Compactness, Surface Area, Wall Area, Roof Area, Overall Height, Orientation, Glazing Area and Glazing Area Distribution.

The analysis was performed using numpy and scikit learn.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso, LassoCV
from sklearn.utils import resample
from scipy.stats import norm

np.random.seed(42)

In [2]:
data = pd.read_excel("ENB2012_data.xlsx")
data.describe()

,X1,X2,X3,X4,X5,X6,X7,X8,Y1,Y2
count,768.000000,768.000000,768.000000,768.000000,768.00000,768.000000,768.000000,768.00000,768.000000,768.000000
mean,0.764167,671.708333,318.500000,176.604167,5.25000,3.500000,0.234375,2.81250,22.307195,24.587760
std,0.105777,88.086116,43.626481,45.165950,1.75114,1.118763,0.133221,1.55096,10.090204,9.513306
min,0.620000,514.500000,245.000000,110.250000,3.50000,2.000000,0.000000,0.00000,6.010000,10.900000
25%,0.682500,606.375000,294.000000,140.875000,3.50000,2.750000,0.100000,1.75000,12.992500,15.620000
50%,0.750000,673.750000,318.500000,183.750000,5.25000,3.500000,0.250000,3.00000,18.950000,22.080000
75%,0.830000,741.125000,343.000000,220.500000,7.00000,4.250000,0.400000,4.00000,31.667500,33.132500
max,0.980000,808.500000,416.500000,220.500000,7.00000,5.000000,0.400000,5.00000,43.100000,48.030000


In [3]:
x = data.drop(columns = ["Y1", "Y2"]) #extract features
y = data["Y1"]                        #extract response
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

### Model Selection

In [4]:
alphas = np.logspace(-6, 0, 100)
#fit lasso using 5-fold CV, making sure to normalize data so that parameters are penalized fairly
lassoCV = LassoCV(cv=5, alphas = alphas, normalize = True, max_iter = 1000000)
reg = lassoCV.fit(x_train, y_train)
r2 = reg.score(x_test, y_test)

print(f"Best model obtained for lambda = {reg.alpha_}, with R2 = {r2:.4f} on test data")

Best model obtained for lambda = 4.328761281083062e-05, with R2 = 0.9121 on test data


Achieving an R2 score of $91\%$, our model is able to explain most of the variance of the reponse from the features alone, without considering higher orders. 

## Confidence intervals and p-values using bootstrap

As the variance of the paramaters have no closed form expression in the case of Lasso, we use bootstrap sampling to estimate these quantities. The bootstrap samples are generated by resampling the training set from earlier, and the models are trained using the best $\lambda$.

In [5]:
best_alpha = reg.alpha_

In [6]:
n = x_train.shape[0]
intercept = []
coef = [] # coefficients of bootstrap models
B = 100 # number of bootstrap samples
for i in range(B):
    model = Lasso(alpha=best_alpha, normalize=True, max_iter = 100000)
    x_boot, y_boot = resample(x_train, y_train, replace=True, n_samples=n)
    model.fit(x_boot, y_boot)
    intercept.append(model.intercept_)
    coef.append(model.coef_)

In [7]:
intercept_array = np.array(intercept)
coef_array = np.array(coef)

intercept_mean = np.mean(intercept_array) # average of intercept across bootstrap samples
intercept_std = np.std(intercept_array) # standard deviation of intercept across bootstrap samples

coef_mean = np.mean(coef_array, axis=0) # average of coefficients across bootstrap samples
coef_std = np.std(coef_array, axis=0) # standard deviation of coefficients across bootstrap samples

#### $95\%$ confidence interval of coefficients:

In [8]:
print(f"intercept: {intercept_mean:.3f} +- {1.95*intercept_std:.3f}")

print(f"X1: {coef_mean[0]:.3f} +- {1.95*coef_std[0]:.3f}")
print(f"X2: {coef_mean[1]:.3f} +- {1.95*coef_std[1]:.3f}")
print(f"X3: {coef_mean[2]:.3f} +- {1.95*coef_std[2]:.3f}")
print(f"X4: {coef_mean[3]:.3f} +- {1.95*coef_std[3]:.3f}")
print(f"X5: {coef_mean[4]:.3f} +- {1.95*coef_std[4]:.3f}")
print(f"X6: {coef_mean[5]:.3f} +- {1.95*coef_std[5]:.3f}")
print(f"X7: {coef_mean[6]:.3f} +- {1.95*coef_std[6]:.3f}")
print(f"X8: {coef_mean[7]:.3f} +- {1.95*coef_std[7]:.3f}")

intercept: 76.940 +- 21.782
X1: -60.027 +- 11.975
X2: -0.073 +- 0.019
X3: 0.053 +- 0.011
X4: -0.020 +- 0.006
X5: 4.107 +- 0.695
X6: -0.026 +- 0.223
X7: 20.004 +- 1.928
X8: 0.213 +- 0.134


#### P-values

The p-values of the coefficients are calculated using the standard deviation of the coefficient estimated using bootstrap sampling. The p-values are calculated with respect to one of the bootstrap samples, the first sample in this case(one experiment). The coefficients are assumed to be normally distributed, by the law of large numbers. 

In [9]:
intercept_scaled = intercept_array[0]/intercept_std
coef_scaled = coef_array[0]/coef_std

p_intercept = 2*(1-norm.cdf(np.abs(intercept_scaled)))
p_coef = 2*(1-norm.cdf(np.abs(coef_scaled)))

In [10]:
print(f"intercept: P={p_intercept:.5f}")

print(f"X1: P={p_coef[0]:.5f}")
print(f"X2: P={p_coef[1]:.5f}")
print(f"X3: P={p_coef[2]:.5f}")
print(f"X4: P={p_coef[3]:.5f}")
print(f"X5: P={p_coef[4]:.5f}")
print(f"X6: P={p_coef[5]:.5f}")
print(f"X7: P={p_coef[6]:.5f}")
print(f"X8: P={p_coef[7]:.5f}")

intercept: P=0.00000
X1: P=0.00000
X2: P=0.00000
X3: P=0.00000
X4: P=0.00000
X5: P=0.00000
X6: P=0.33665
X7: P=0.00000
X8: P=0.00001


Comparing the confidence intervals and p-values, we see that all but feature X6 is highly significant. Allthough some of the coefficients have quite wide confidence intervals, such as the intercept and X1, they are still significant as their magnitude are relativly bigger than the width of their confidence interval. X6's confidence interval, on the other hand, contain 0. This indicates that the orientation of the room have very little explanatory power with respect to the heat load of the room.